In [23]:
import pandas as pd
import matplotlib.pyplot as plt
import ast
from googletrans import Translator
import re
from nltk.stem import WordNetLemmatizer
from deep_translator import GoogleTranslator
from langdetect import detect

In [2]:
df = pd.read_parquet('/export/data_ml4ds/AI4U/Datasets/ResearchPortal/20240321/parquet/projects.parquet')

In [3]:
df_keywords = df.dropna(subset=['Keywords'])[['actID', 'Keywords']]

In [4]:
# ME QUEDO CON TODAS LAS KEYWORDS EN UNA LISTA
all_keywords = []

for keywords_list in df_keywords['Keywords']:
    for keywords in keywords_list:
        # Dividir cada string en una lista de elementos usando ';' como separador
        split_keywords = keywords.split('; ')
        
        if ';' in split_keywords:
            split_keywords = split_keywords.split(';')
            
        # Extender la lista all_keywords con los elementos de cada lista resultante
        all_keywords.extend(split_keywords)

print(len(all_keywords)) #10874

# ELIMINO LAS DUPLICADAS PARA QUEDARME SOLO CON LAS UNICAS
unique_keywords = list(set(all_keywords))
print(len(unique_keywords))

10874
7160


In [5]:
# Función para dividir elementos que contienen listas internas en elementos individuales
def flatten_word_list(word_list):
    flattened_list = []
    for word in word_list:
        if ',' in word or ';' in word:
            # Separar las palabras utilizando ',' o ';' como delimitadores
            sub_words = re.split(r'[;,]\s*', word)
            flattened_list.extend(sub_words)
        else:
            flattened_list.append(word)
    return flattened_list

# Aplicar la función para aplanar la lista de palabras
#flattened_words = flatten_word_list(unique_lemmatized_words)
unique_keywords
flattened_words = flatten_word_list(unique_keywords)

# Imprimir la lista aplanada
print(len(flattened_words))

8587


In [6]:
keyword_translations = {}

# Iterar sobre las palabras y traducir al inglés si es necesario
for palabra in flattened_words:
    
    if GoogleTranslator(source='auto', target='en').translate(palabra) != palabra:
        traduccion = GoogleTranslator(source='auto', target='en').translate(palabra)
        keyword_translations[palabra] = traduccion
    else:
        keyword_translations[palabra] = palabra

#keyword_translations
keyword_translations_df = pd.DataFrame(list(keyword_translations.items()), columns=['Palabra Original', 'Traducción'])
keyword_translations_df.to_csv('traducciones.csv', index=False)

In [7]:
keyword_translations_df[

,Palabra Original,Traducción
0,,
1,fft,fft
2,historia de la geografía,history of geography
3,problemas no locales,non-local problems
4,privacidad,privacy
...,...,...
7800,calificación,qualification
7801,rams,rams
7802,informe pericial,expert report
7803,transición española,Spanish transition


In [72]:
# Ejecutar 
df = pd.read_parquet('/export/data_ml4ds/AI4U/Datasets/ResearchPortal/20240321/parquet/projects.parquet')
df_traducciones = pd.read_csv('traducciones.csv')
traducciones = df_traducciones.set_index('Palabra Original')['Traducción'].to_dict()

In [73]:
def traducir_keywords(keywords_list, diccionario_traducciones):
    
    if keywords_list:    
        
        # Dividir cada string en una lista de elementos usando ';' o ',' como separador
        keywords_list = re.sub(r'[\[\]\']', '', keywords_list)
        
        # Reemplazar diferentes separadores por un único separador estándar
        keywords_list = re.sub(r'\s*;\s*', '; ', keywords_list)  # ' ; ' o ';' -> '; '
        keywords_list = re.sub(r'\s*,\s*', ', ', keywords_list)  # ' , ' o ',' -> ', '
        
        # Dividir las palabras clave utilizando '; ' o ', ' como separadores
        split_keywords = re.split(r'; |, ', keywords_list)    

        # Traducir cada palabra clave utilizando el diccionario de traducciones
        translated_keywords = [diccionario_traducciones.get(keyword, keyword) for keyword in split_keywords]

        # Unir las palabras clave traducidas en un solo string usando '; ' como separador
        return ', '.join(translated_keywords)
    else:
        return None

df['Keywords'] = df['Keywords'].astype(str).apply(lambda x: re.sub(r'[\[\]\']', '', x))    

# Aplicar la función a cada fila de la columna 'Keywords'
df['Translated_Keywords'] = df['Keywords'].apply(lambda x: traducir_keywords(x, traducciones))

df['Keywords'] = df['Keywords'].replace('None', None)
df['Translated_Keywords'] = df['Translated_Keywords'].replace('None', None)

df_keywords_def = df.dropna(subset=['Keywords'])[['actID', 'Keywords', 'Translated_Keywords']]

In [80]:
#df_keywords_def.to_csv('keywords_translated_def.csv', index=False)
df

,actID,Title,StartYear,EndYear,Funding_Entity,Reference,Type,Keywords,resumen,Translated_Keywords
0,act527937,Personalized Planning in Radio Therapy Through...,2020,2024,AECC (ASOCIACION ESPAÑOLA CONTRA EL CANCER) JU...,PERME20731PASC,Private Sector Competitive Project,None,None,None
1,act536663,NOVExport un hub acelerador de exportaciones p...,2021,2022,EUROPEAN COMMISSION RESEARCH EXECUTIVE AGENCY,SOE3/P2/F0970,European Research Project,None,None,None
2,act537425,Hacia unas plantas solares de concentración co...,2020,2022,AGENCIA ESTATAL DE INVESTIGACION (AEI),EIN2020-112442,Complementary National Fundings for Research A...,energía solar de concentración; torre solar; a...,ESTE PROYECTO FORMATIVO OFRECERA A INVESTIGADO...,"concentrating solar energy, solar tower, high ..."
3,act537453,Una Inteligencia Artificial Visual Centrada en...,2021,2025,AGENCIA ESTATAL DE INVESTIGACION (AEI),PID2020-118504GB-I00,National Research Project,inteligencia artificial; visión artificial; in...,HOY EN DIA LA INTELIGENCIA ARTIFICIAL (IA) HA ...,"artificial intelligence, artificial vision, hu..."
4,act537480,Hacia una mejor comprensión de las bases bioló...,2021,2025,AGENCIA ESTATAL DE INVESTIGACION (AEI),PID2020-119792RB-I00,National Research Project,enfermedades raras; terapia génica; aptámeros;...,LA EPIDERMOLISIS BULLOSA (EB) COMPRENDE UN GRU...,"rare diseases, gene therapy, aptamer, genoderm..."
...,...,...,...,...,...,...,...,...,...,...
5620,act562687,Contrato Asesoría Gesnaer,2024,2025,GESNAER CONSULTING S.L.N.E.,None,Technical Assessment and Assistance Contract,contrato asesoria; consultoría,None,"consultancy contract, consultancy"
5621,act562713,AI para la construcción del Digital Energy Twin,2021,2022,"BALANTIA CONSULTORES, S.L.",None,Technical Assessment and Assistance Contract,None,None,None
5622,act562714,Estudio sobre la situación de la profesión del...,2024,2024,FUNDACION SGAE,None,R+D Contract,industria audiovisual en españa; audiovisual; ...,None,"audiovisual industry in Spain, audiovisual, di..."
5623,act562715,Contrato para la prestación de Servicios de As...,2021,2026,TELYNET SA,None,Technical Assessment and Assistance Contract,machine learning; estadística avanzada,None,"machine learning, advanced statistics"


In [78]:
# SOBREESCRIBIR EN LA TABLA PROJECTS PARA AÑADIR LAS TRADUCCIONES
df.to_parquet('/export/data_ml4ds/AI4U/Datasets/ResearchPortal/20240321/processed_parquets/projects_translated_keywords.parquet')